# Set up Web Driver

In [49]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [50]:
wd = webdriver.Chrome("/opt/anaconda3/chromedriver-Darwin")
wd.get("https://clubhub.site")

# Get Live Room Information

In [51]:
from datetime import datetime
import time
import re
import pandas as pd 

In [44]:
cnt_rooms_raw = wd.find_element_by_xpath("""//*[@id="app"]/div[1]/main/div/div[2]/div/h3/span""").text

cnt_rooms = int(re.search(r'\d+', cnt_rooms_raw).group())
cnt_rooms

78

In [45]:
### UTC TIME
capture_time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

In [46]:
all_rooms = []
for x in range(cnt_rooms):
    xpath = """//*[@id="app"]/div[1]/main/div/div[3]/a[""" + str(x+1) + "]" # simplified
    if len(wd.find_element_by_xpath(xpath).text.split("\n")) == 6:
        room_name = wd.find_element_by_xpath(xpath).text.split("\n")[0]
        club_name = re.sub('from (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[1])
        speaker_cnt = re.sub('💬 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[3])
        joiner_cnt = re.sub('👥 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[4])
        duration = re.sub('🕒 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[5])
    elif len(wd.find_element_by_xpath(xpath).text.split("\n")) == 5:
        room_name = wd.find_element_by_xpath(xpath).text.split("\n")[0]
        club_name = re.sub('from (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[1])
        speaker_cnt = re.sub('💬 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[2])
        joiner_cnt = re.sub('👥 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[3])
        duration = re.sub('🕒 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[4])
    elif len(wd.find_element_by_xpath(xpath).text.split("\n")) == 4:
        room_name = wd.find_element_by_xpath(xpath).text.split("\n")[0]
        club_name = ""
        speaker_cnt = re.sub('💬 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[1])
        joiner_cnt = re.sub('👥 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[2])
        duration = re.sub('🕒 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[3])
    room_details = {
        'capture_time' : capture_time,
        'room_name' : room_name,
        'club_name' : club_name,
        'speaker_cnt' : speaker_cnt,
        'joiner_cnt' : joiner_cnt,
        'duration' : duration
    }
    all_rooms.append(room_details)

In [47]:
room_df = pd.DataFrame(all_rooms)
room_df

,capture_time,room_name,club_name,speaker_cnt,joiner_cnt,duration
0,29/04/2021 07:00:43,ROLL TIDE to the NFL,NFL 🏠,6,925,00:35:08
1,29/04/2021 07:00:43,JIM KWIK & Natasha Grano 🧠⚡️Become LIMITLESS,TALK CLUB 🏠,41,599,02:01:14
2,29/04/2021 07:00:43,Imogen Heap x Endlesss | Genesis NFT Drop LIVE!,🤓TALK NERDY TO ME🦾🚀🩺🕹♻️🧬🎞🪐 🏠,25,543,02:00:43
3,29/04/2021 07:00:43,❤️DATING GAME + SHOOT UR SHOT w/ the Sexiest S...,SINGLE IN THE CITY 🏠,87,347,20:56:09
4,29/04/2021 07:00:43,Atlanta is the Best Show Out PERIOD!!!,COMEDY CLUB 🏠,41,253,01:49:45
...,...,...,...,...,...,...
73,29/04/2021 07:00:43,🔥🎶Hump Day Vibes 🎶🔥,THE HOUSE RADIO 🏠,2,14,02:34:42
74,29/04/2021 07:00:43,NASCAR Talk: Talladega/Kansas,NASCAR TALK. 🏠,12,13,00:25:53
75,29/04/2021 07:00:43,Working 😩,SADBOIS R US 🏠,12,13,04:57:06
76,29/04/2021 07:00:43,Public Speaking Tips from TEDx Speakers + Coac...,BEAUTY GURU’S CONNECT 🏠,2,11,00:02:50


In [48]:
room_df.to_csv('Downloads/clubhouse_'+str(datetime.now())+'.csv')

# Set up crawling

In [85]:
def get_room_details(url):
    wd = webdriver.Chrome("/opt/anaconda3/chromedriver-Darwin")
    wd.get(url)
    time.sleep(15)
    
    cnt_rooms_raw = wd.find_element_by_xpath("""//*[@id="app"]/div[1]/main/div/div[2]/div/h3/span""").text
    cnt_rooms = int(re.search(r'\d+', cnt_rooms_raw).group())
    
    capture_time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    
    all_rooms = []
    for x in range(cnt_rooms):
        xpath = """//*[@id="app"]/div[1]/main/div/div[3]/a[""" + str(x+1) + "]" # simplified
        if len(wd.find_element_by_xpath(xpath).text.split("\n")) == 6:
            room_name = wd.find_element_by_xpath(xpath).text.split("\n")[0]
            club_name = re.sub('from (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[1])
            speaker_cnt = re.sub('💬 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[3])
            joiner_cnt = re.sub('👥 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[4])
            duration = re.sub('🕒 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[5])
        elif len(wd.find_element_by_xpath(xpath).text.split("\n")) == 5:
            room_name = wd.find_element_by_xpath(xpath).text.split("\n")[0]
            club_name = re.sub('from (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[1])
            speaker_cnt = re.sub('💬 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[2])
            joiner_cnt = re.sub('👥 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[3])
            duration = re.sub('🕒 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[4])
        elif len(wd.find_element_by_xpath(xpath).text.split("\n")) == 4:
            room_name = wd.find_element_by_xpath(xpath).text.split("\n")[0]
            club_name = ""
            speaker_cnt = re.sub('💬 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[1])
            joiner_cnt = re.sub('👥 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[2])
            duration = re.sub('🕒 (.*?)', '', wd.find_element_by_xpath(xpath).text.split("\n")[3])
        room_details = {
            'capture_time' : capture_time,
            'room_name' : room_name,
            'club_name' : club_name,
            'speaker_cnt' : speaker_cnt,
            'joiner_cnt' : joiner_cnt,
            'duration' : duration
        }
        all_rooms.append(room_details)
    room_df = pd.DataFrame(all_rooms)
    room_df.to_csv('Downloads/clubhouse_'+str(datetime.now())+'.csv')
    print(str(datetime.now()))
    
    time.sleep(1800) #sleep for 30mins
    return room_df

In [91]:
timeout = time.time() + 60*60*8   # 8 hour from now
timeout

1619983024.371916

In [92]:
while True:
    test = 0
    if test == 5 or time.time() > timeout:
        break
    else:
        get_room_details('https://clubhub.site')
    test = test - 1

2021-05-02 19:17:31.865395
2021-05-02 19:47:56.042674
2021-05-02 20:18:21.014160
2021-05-02 20:48:45.847539
2021-05-02 21:19:11.862440
2021-05-02 21:49:36.838868
2021-05-02 22:20:02.883554
2021-05-02 22:50:30.048720
2021-05-02 23:20:56.050555
2021-05-02 23:51:21.339167


KeyboardInterrupt: 